# Vertex AI Training and Endpoints
### Predict the fuel efficiency of a vehicle

### Environment setup

In [1]:
# Setting project
!gcloud config set project itam-dpa-2022

Updated property [core/project].


In [2]:
# Enable all the APIs that we are going to use
! gcloud services enable compute.googleapis.com \
                       containerregistry.googleapis.com \
                       aiplatform.googleapis.com \
                       cloudbuild.googleapis.com

Operation "operations/acat.p2-171673689672-0e3b5b44-40d3-4641-91b2-98d8da6d83c9" finished successfully.


In [3]:
# Creating a new bucket (in a specific region)
!gsutil mb -l us-central1 gs://itam-dpa-2022-model2

Creating gs://itam-dpa-2022-model2/...
ServiceException: 409 A Cloud Storage bucket named 'itam-dpa-2022-model2' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


### Modeling

In [4]:
import numpy as np
import pandas as pd
import pathlib
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.8.0


In [5]:
BUCKET = "gs://itam-dpa-2022-model2"

In [6]:
# Import data it using pandas

"""## The Auto MPG dataset

The dataset is available from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/).

### Get the data
First download the dataset.
"""

dataset_path = "https://storage.googleapis.com/io-vertex-codelab/auto-mpg.csv"
dataset = pd.read_csv(dataset_path, na_values = "?")
dataset.pop("car name")
dataset.shape

(398, 8)

In [7]:
dataset.tail()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
393,27.0,4,140.0,86.0,2790,15.6,82,1
394,44.0,4,97.0,52.0,2130,24.6,82,2
395,32.0,4,135.0,84.0,2295,11.6,82,1
396,28.0,4,120.0,79.0,2625,18.6,82,1
397,31.0,4,119.0,82.0,2720,19.4,82,1


In [8]:
# Checking NAs
dataset.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model year      0
origin          0
dtype: int64

In [9]:
# Dropping those values
dataset = dataset.dropna()
print(dataset.shape)

(392, 8)


In [10]:
dataset

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,8,307.0,130.0,3504,12.0,70,1
1,15.0,8,350.0,165.0,3693,11.5,70,1
2,18.0,8,318.0,150.0,3436,11.0,70,1
3,16.0,8,304.0,150.0,3433,12.0,70,1
4,17.0,8,302.0,140.0,3449,10.5,70,1
...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,1
394,44.0,4,97.0,52.0,2130,24.6,82,2
395,32.0,4,135.0,84.0,2295,11.6,82,1
396,28.0,4,120.0,79.0,2625,18.6,82,1


### Feature Engineering

In [11]:
# One-hot-encoding 
dataset['origin'] = dataset['origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,Europe,Japan,USA
393,27.0,4,140.0,86.0,2790,15.6,82,0,0,1
394,44.0,4,97.0,52.0,2130,24.6,82,1,0,0
395,32.0,4,135.0,84.0,2295,11.6,82,0,0,1
396,28.0,4,120.0,79.0,2625,18.6,82,0,0,1
397,31.0,4,119.0,82.0,2720,19.4,82,0,0,1


In [12]:
# Split data (X's)
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [13]:
# Calculate statistics for normalization
train_stats = train_dataset.describe()
train_stats.pop("mpg")
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
cylinders,314.0,5.477707,1.699788,3.0,4.00,4.0,8.00,8.0
displacement,314.0,195.318471,104.331589,68.0,105.50,151.0,265.75,455.0
horsepower,314.0,104.869427,38.096214,46.0,76.25,94.5,128.00,225.0
weight,314.0,2990.251592,843.898596,1649.0,2256.50,2822.5,3608.00,5140.0
acceleration,314.0,15.559236,2.789230,8.0,13.80,15.5,17.20,24.8
model year,314.0,75.898089,3.675642,70.0,73.00,76.0,79.00,82.0
Europe,314.0,0.178344,0.383413,0.0,0.00,0.0,0.00,1.0
Japan,314.0,0.197452,0.398712,0.0,0.00,0.0,0.00,1.0
USA,314.0,0.624204,0.485101,0.0,0.00,1.0,1.00,1.0


In [14]:
# Getting labels (Y's)
train_labels = train_dataset.pop('mpg')
test_labels = test_dataset.pop('mpg')

In [15]:
# Normalizing dataset
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [16]:
normed_train_data.head()

,cylinders,displacement,horsepower,weight,acceleration,model year,Europe,Japan,USA
146,-0.869348,-1.009459,-0.784052,-1.025303,-0.379759,-0.516397,-0.465148,-0.495225,0.774676
282,-0.869348,-0.530218,-0.442811,-0.118796,0.624102,0.843910,-0.465148,-0.495225,0.774676
69,1.483887,1.482595,1.447140,1.736877,-0.738281,-1.060519,-0.465148,-0.495225,0.774676
378,-0.869348,-0.865687,-1.099044,-1.025303,-0.308055,1.660094,-0.465148,-0.495225,0.774676
331,-0.869348,-0.942365,-0.994047,-1.001603,0.875068,1.115971,-0.465148,2.012852,-1.286751


### About feature engineering and data leakeage
The statistics used to normalize the inputs here (mean and standard deviation) need to be applied to any other data that is fed to the model, along with the one-hot encoding that we did earlier.  That includes the test set as well as live data when the model is used in production.


# Training a model

Let's build our model. Here, we'll use a `Sequential` model with two densely connected hidden layers, and an output layer that returns a single, continuous value. The model building steps are wrapped in a function, `build_model`, since we'll create a second model, later on.

In [17]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
  return model

In [18]:
len(train_dataset.keys())

9

In [19]:
# Instantiating the model
model = build_model()
model.summary()

2022-03-24 20:15:10.602199: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-03-24 20:15:10.602276: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-24 20:15:10.602321: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vm-1a1132b4-6adb-45c9-b1ce-76bd1b8b9bc5): /proc/driver/nvidia/version does not exist
2022-03-24 20:15:10.604838: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                640       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,865
Trainable params: 4,865
Non-trainable params: 0
_________________________________________________________________


In [20]:
EPOCHS = 1000

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

early_history = model.fit(normed_train_data, train_labels, 
                    epochs=EPOCHS, validation_split = 0.2, 
                    callbacks=[early_stop])

Epoch 1/1000
8/8 [==============================] - 2s 124ms/step - loss: 568.5611 - mae: 22.5956 - mse: 568.5611 - val_loss: 554.9382 - val_mae: 22.3001 - val_mse: 554.9382
Epoch 2/1000
8/8 [==============================] - 0s 14ms/step - loss: 511.8723 - mae: 21.3827 - mse: 511.8723 - val_loss: 495.7735 - val_mae: 20.9962 - val_mse: 495.7735
Epoch 3/1000
8/8 [==============================] - 0s 15ms/step - loss: 456.5231 - mae: 20.1230 - mse: 456.5231 - val_loss: 433.0129 - val_mae: 19.5266 - val_mse: 433.0129
Epoch 4/1000
8/8 [==============================] - 0s 14ms/step - loss: 398.1742 - mae: 18.7032 - mse: 398.1742 - val_loss: 368.4419 - val_mae: 17.8864 - val_mse: 368.4419
Epoch 5/1000
8/8 [==============================] - 0s 14ms/step - loss: 336.7921 - mae: 17.0905 - mse: 336.7921 - val_loss: 299.9591 - val_mae: 15.9890 - val_mse: 299.9591
Epoch 6/1000
8/8 [==============================] - 0s 9ms/step - loss: 273.3950 - mae: 15.2566 - mse: 273.3950 - val_loss: 233.9944 -

In [21]:
# Export model and save to GCS
model.save(BUCKET + '/mpg/model')

2022-03-24 20:15:22.680152: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: gs://itam-dpa-2022-model2/mpg/model/assets


### Packaging everything in a Docker container

In [22]:
# Creating some files
!mkdir mpg && cd mpg && mkdir trainer

mkdir: cannot create directory ‘mpg’: File exists


In [23]:
%%writefile /home/jupyter/model2/mpg/Dockerfile
FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-3
WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.train"]

Overwriting /home/jupyter/model2/mpg/Dockerfile


In [24]:
%%writefile /home/jupyter/model2/mpg/trainer/train.py
# Training script
BUCKET = "gs://itam-dpa-2022-model2"

import numpy as np
import pandas as pd
import pathlib
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

#importing data
dataset_path = "https://storage.googleapis.com/io-vertex-codelab/auto-mpg.csv"
dataset = pd.read_csv(dataset_path, na_values = "?")
dataset.pop("car name")
dataset.tail()


#checking na's
dataset.isna().sum()

#dropping na's
dataset = dataset.dropna()

#feature engineering
dataset['origin'] = dataset['origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

#spliting data
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

#calculating statistics for normalization
train_stats = train_dataset.describe()
train_stats.pop("mpg")
train_stats = train_stats.transpose()
train_stats

#getting labels
train_labels = train_dataset.pop('mpg')
test_labels = test_dataset.pop('mpg')

#normalize data
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)


#build the model 
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
  return model

model = build_model()
model.summary()

EPOCHS = 1000
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
early_history = model.fit(normed_train_data, train_labels, epochs=EPOCHS, validation_split = 0.2, callbacks=[early_stop])

# Export model and save to GCS
model.save(BUCKET + '/mpg/model')

Overwriting /home/jupyter/model2/mpg/trainer/train.py


# ML Engineer Work: who is responsible for the next part?
In an organization, it's common to have one team or individual in charge of building the model, and a different team in charge of deploying it. The steps we'll go through here will show you how to take a model that's already been trained and deploy it for prediction.

### Building the Docker image and saving into the Container Registry 

**Note**: 
1) You have to enable manually the Cloud Build API in https://console.cloud.google.com/marketplace/product/google/cloudbuild.googleapis.com
2) You have to enable service account permissions in https://console.cloud.google.com/cloud-build/settings/service-account?project=itam-dpa-2022

In [25]:
# Submitting the a docler build job to Google Cloud Build --be carefull to point to the Dockerfile
!gcloud builds submit --tag gcr.io/itam-dpa-2022/mpg:v4 ./mpg/

Creating temporary tarball archive of 4 file(s) totalling 4.1 KiB before compression.
Uploading tarball of [./mpg/] to [gs://itam-dpa-2022_cloudbuild/source/1648152930.753758-f911276dbea8476589be2e2bedaf999a.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/itam-dpa-2022/locations/global/builds/73f85638-8bf9-4cc9-a664-586847907fbc].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/73f85638-8bf9-4cc9-a664-586847907fbc?project=171673689672].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "73f85638-8bf9-4cc9-a664-586847907fbc"

FETCHSOURCE
Fetching storage object: gs://itam-dpa-2022_cloudbuild/source/1648152930.753758-f911276dbea8476589be2e2bedaf999a.tgz#1648152930951771
Copying gs://itam-dpa-2022_cloudbuild/source/1648152930.753758-f911276dbea8476589be2e2bedaf999a.tgz#1648152930951771...
/ [1 files][  1.6 KiB/  1.6 KiB]                                                
Operation completed over 1 objects/1.

# Submit a training job in GCP using the UI

https://console.cloud.google.com/vertex-ai/training/training-pipelines?project=itam-dpa-2022
https://cloud.google.com/vertex-ai/docs/training/custom-training

# Endpoint Deployment

In [26]:
from google.cloud import aiplatform

# Create a model resource from public model assets
model = aiplatform.Model.upload(
    display_name="itam_dpa_2022_mpg_regression",
    artifact_uri="gs://itam-dpa-2022-model2/mpg/model/",
    serving_container_image_uri="gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest"
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/171673689672/locations/us-central1/models/335258687494946816/operations/2945849555008094208
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/171673689672/locations/us-central1/models/335258687494946816
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/171673689672/locations/us-central1/models/335258687494946816')


In [27]:
model

resource name: projects/171673689672/locations/us-central1/models/335258687494946816

In [28]:
# Deploy the above model to an endpoint
endpoint = model.deploy(
    machine_type="n1-standard-4"
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/171673689672/locations/us-central1/endpoints/1769254946579939328/operations/4661721013036253184
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/171673689672/locations/us-central1/endpoints/1769254946579939328
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/171673689672/locations/us-central1/endpoints/1769254946579939328')
INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/171673689672/locations/us-central1/endpoints/1769254946579939328
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/171673689672/locations/us-central1/endpoints/1769254946579939328/operations/7602571569709187072
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/17167368967

In [29]:
endpoint

resource name: projects/171673689672/locations/us-central1/endpoints/1769254946579939328

# Making a prediction from an End-point

In [30]:
from google.cloud import aiplatform

endpoint = aiplatform.Endpoint(
    endpoint_name="projects/171673689672/locations/us-central1/endpoints/1769254946579939328"
)

# A test example we'll send to our model for prediction
test_mpg = [1.4838871833555929, 1.8659883497083019, 2.234620276849616, 1.0187816540094903, -2.530890710602246, -1.6046416850441676, -0.4651483719733302, -0.4952254087173721, 0.7746763768735953]

response = endpoint.predict([test_mpg])

print('API response: ', response)

print('Predicted MPG: ', response.predictions[0][0])

API response:  Prediction(predictions=[[16.4352493]], deployed_model_id='6716793388781273088', explanations=None)
Predicted MPG:  16.4352493


# Reference
For more information visit:
- [codelab](https://codelabs.developers.google.com/codelabs/vertex-ai-custom-models#0)
- [video tutorial](https://youtu.be/aB2OxnyfP0c)

Thanks Sarah Robinson